In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customers-100.csv to customers-100.csv


In [ ]:
import pandas as pd
import random

In [ ]:
# Load the CSV
df = pd.read_csv("/content/customers-100.csv")

# Normalize column name if needed
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

df.head()

,First_Name,Last_Name,Company,City,Country,Phone,Email
0,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,zunigavanessa@smith.info
1,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,vmata@colon.com
2,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,-1199,beckycarr@hogan.com
3,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,stanleyblackwell@benson.org
4,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,colinalvarado@miles.net


In [ ]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t
        self.leaf = leaf
        self.keys = []
        self.children = []

In [ ]:
class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, True)
        self.t = t
        self.comparisons = 0

    def search(self, k, node=None):
        if node is None:
            node = self.root

        i = 0

        while i < len(node.keys) and k > node.keys[i][0]:
            self.comparisons += 1
            i += 1

        if i < len(node.keys):
            self.comparisons += 1
            if node.keys[i][0] == k:
                return node.keys[i]

        if node.leaf:
            return None

        return self.search(k, node.children[i])

    def split_child(self, parent, i):
        t = self.t
        node = parent.children[i]
        new_node = BTreeNode(t, node.leaf)

        parent.keys.insert(i, node.keys[t - 1])
        parent.children.insert(i + 1, new_node)

        new_node.keys = node.keys[t:(2 * t - 1)]
        node.keys = node.keys[0:t - 1]

        if not node.leaf:
            new_node.children = node.children[t:(2 * t)]
            node.children = node.children[:t]

    def insert(self, k):
        root = self.root
        if len(root.keys) == (2 * self.t - 1):
            new_node = BTreeNode(self.t, False)
            new_node.children.append(self.root)
            self.root = new_node
            self.split_child(new_node, 0)
            self._insert_non_full(new_node, k)
        else:
            self._insert_non_full(root, k)

    def _insert_non_full(self, node, k):
        if node.leaf:
            node.keys.append(k)
            node.keys.sort(key=lambda x: x[0])
        else:
            i = len(node.keys) - 1
            while i >= 0 and k[0] < node.keys[i][0]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == (2 * self.t - 1):
                self.split_child(node, i)
                if k[0] > node.keys[i][0]:
                    i += 1

            self._insert_non_full(node.children[i], k)

In [ ]:
btree = BTree(t=3)
built_in_list = []
hashmap = {}

for _, row in df.iterrows():
    key = row.First_Name
    data = row.to_dict()

    btree.insert((key, data))   # Inserts tuple
    built_in_list.append(data)   # Store in List
    hashmap[key] = data         # Store in dictionary

In [ ]:
names = df.First_Name.tolist()
search_list = random.sample(names, 100)

In [ ]:
btree.comparisons = 0
btree_results = []

for name in search_list:
    btree.search(name)
btree_comps = btree.comparisons

In [ ]:
list_total_comparisons = 0

# List Search (Linear Scan)
for key in search_list:
    comps = 0
    for item in built_in_list:
        comps += 1
        if item['First_Name'] == key:
            break
    list_total_comparisons += comps

In [ ]:
hashmap_comparisons = 0

for name in search_list:
    hashmap_comparisons += 1  # 1 comparison for key lookup
    _ = hashmap.get(name, None)

In [ ]:
print("-" * 40)
print(f"Total Records Searched: {len(search_list)}")
print("-" * 40)
print("B-Tree Comparisons for 100 searches:", btree_comps)
print("List Comparisons for 100 searches:", list_total_comparisons)
print("HashMap Comparisons for 100 searches:", hashmap_comparisons)

----------------------------------------
Total Records Searched: 100
----------------------------------------
B-Tree Comparisons for 100 searches: 691
List Comparisons for 100 searches: 4814
HashMap Comparisons for 100 searches: 100
